# API HH
https://api.hh.ru/openapi/redoc#tag/Poisk-vakansij/operation/get-vacancies - описание api

In [ ]:
import requests

In [ ]:
# Тестовые пример
# profession = "Программист на Java"
# params = {"text": profession, "page": 0,"per_page": 5}
# url = "https://api.hh.ru/vacancies"
# response = requests.get(url, params=params)
# data = response.json()
# data["items"][0]

In [ ]:
# Список доступных хостов
hosts = [
    "hh.ru", "rabota.by", "hh1.az", "hh.uz", "hh.kz",
    "headhunter.ge", "headhunter.kg"
]

In [ ]:
# url_get_vacancies = "https://api.hh.ru/vacancies"
def get_vacancies(text, per_page = 100, host ="hh.ru"):
  url_get_vacancies = f"https://api.{host}/vacancies"

  vacancies = []
  print(text)
  for page in range(20):
      params = {
          "text": text,
          "host": host,
          "page": page,
          "per_page": per_page}
      try:
          response = requests.get(url_get_vacancies, params=params)
          if response.status_code == 200:
              data = response.json()
              vacancies_per_page = data.get("items", [])
              if not vacancies_per_page:
                  break
              vacancies.extend(vacancies_per_page)
          else:
              break
      except requests.RequestException as e:
              print(f"Ошибка при запросе вакансий: {e}")
              break
  return vacancies

In [ ]:
HOST_URLS = {
    "hh.ru": "https://api.hh.ru/vacancies/{}",
    "rabota.by": "https://api.rabota.by/vacancies/{}",
    "hh1.az": "https://api.hh1.az/vacancies/{}",
    "hh.uz": "https://api.hh.uz/vacancies/{}",
    "hh.kz": "https://api.hh.kz/vacancies/{}",
    "headhunter.ge": "https://api.headhunter.ge/vacancies/{}",
    "headhunter.kg": "https://api.headhunter.kg/vacancies/{}"
}
#можете оптимизировать, если хотите)

In [ ]:
url_get_full_vacancy_by_id = "https://api.hh.ru/vacancies/{vacancy_id}"
def get_vacancy_by_id(vacancy_id, host ="hh.ru"):
    """
    Получение вакансии по ID с учетом выбранного хоста

    :param vacancy_id: ID вакансии
    :param host: хост (по умолчанию hh.ru)
    :return: JSON с информацией о вакансии или пустой список
    """
    if host not in HOST_URLS:
        print(f"Неподдерживаемый хост: {host}")
        return []
    # url = url_get_full_vacancy_by_id.format(vacancy_id=vacancy_id)
    # Формируем URL для конкретного хоста
    url = HOST_URLS[host].format(vacancy_id)
    params = {"host": host}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка при получении вакансии с ID {vacancy_id}: {response.status_code} {response.text}")
        return []

In [ ]:
def get_full_vacancies(vacancies, host="hh.ru", limit = 100):
  """
    Получение полной информации о вакансиях

    :param vacancies: список вакансий
    :param host: хост (по умолчанию hh.ru)
    :param limit: максимальное количество вакансий для обработки
    :return: список полных вакансий
    """
  print("start get_full_vacancies")
  print(limit)
  print(host)
  full_vacancies = []
  for v in vacancies[:min(limit, len(vacancies))]:
      vac = get_vacancy_by_id(v["id"], host)
      if not vac:
          break
      full_vacancies.append(vac)
  return full_vacancies

In [ ]:
# получить ссылку на страницу
def get_url(text, host = "hh.ru"):
    # Генерация URL для поиска вакансий
    return f"https://{host}/search/vacancy?text={text}"

In [ ]:
# vacancies = get_vacancies("Программист на Java")
# print(len(vacancies))

In [ ]:
# full_vanacies = get_full_vacancies(vacancies)
# print(len(full_vanacies))

# БД

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 34.7 MB/s eta 0:00:00


In [ ]:
import mysql.connector
from mysql.connector import Error

In [ ]:
def get_db_connection():
    return mysql.connector.connect(
        host="host",
        user="user",
        password="pass",
        database="db"
    )

In [ ]:
db_connection = None

In [ ]:
def log_message_to_db(message, host, vacancy_description):
    if db_connection is None:
        return
    try:
        cursor = db_connection.cursor()
        insert_query = """
        INSERT INTO telegram_messages (message_id, user_id, first_name, username, message_date, text, host)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(
            insert_query,
            (
                message.message_id,
                message.from_user.id,
                message.from_user.first_name,
                message.from_user.username,
                datetime.utcfromtimestamp(message.date),
                vacancy_description,
                host
            ),
        )
        db_connection.commit()
    except Error as e:
        print(f"Ошибка при сохранении сообщения: {e}")

# ANALYZE

Что хотим получить в итоге?
1. Требования
- топ требований (например, стек технологий)
- опыт работы
- дополнителньые требования какие-то
2. Обязанности
- какие основные обязанности (собрать все обязанности, объеденить)
3. Условия труда
- Какой режим (онлайн/оффлайн/гибрид, какой график работы)
- Условия труда (что предлагают, релокейт, плюшки)
- В каком месте сколько вакансий (например, в москве 100 вакансий, в перми 20 вакансий)
- ЗП (диапазон зарплат, график распределния зп, сравнеие по регионам, средняя зп)
4. Идеальный кандидат (Собрать все данные и сделать из них "идеального кандидата", который подходит 90% вакансий.

Вот какие-то короткие данные хочется получичить в качестве сообщения от тг бота текстом. Какие-то большие данные, подробности хочется получить в pdf файле. Например, все графики можно отправить в pdf. В сообщении просто, например, идеального кандидата

## Excel

In [ ]:
pip install openpyxl matplotlib

In [ ]:
import matplotlib.pyplot as plt
import io
import os
import openpyxl
from openpyxl.drawing.image import Image
from io import BytesIO
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import random
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Font, Border, Side

In [ ]:
# Пример сохранения графика
# def generate_graph():
#     # Пример: создаем случайные данные для графика
#     x = [1, 2, 3, 4, 5]
#     y = [random.randint(1, 10) for _ in x]

#     fig, ax = plt.subplots()
#     ax.plot(x, y, label='Пример данных', color='blue')
#     ax.set_title('Пример графика')
#     ax.set_xlabel('X')
#     ax.set_ylabel('Y')
#     ax.legend()

#     # Сохраняем график в памяти
#     img_stream = BytesIO()
#     canvas = FigureCanvas(fig)
#     canvas.print_png(img_stream)
#     img_stream.seek(0)

#     return img_stream

# # Функция для генерации Excel файла
# def generate_excel(chat_id, text):
#     # Создаем новый Excel файл
#     wb = Workbook()
#     ws = wb.active
#     ws.title = "Вакансии"

#     # Добавляем данные (например, сюда можно добавить информацию о вакансиях)
#     ws.append(["ID вакансии", "Название", "Зарплата"])

#     # Генерация графика
#     img_stream = generate_graph()

#     # Добавление графика в Excel файл
#     img = Image(img_stream)
#     img.anchor = 'E5'  # Позиция графика на листе
#     ws.add_image(img)

#     # Уникальное имя файла
#     file_name = f"{chat_id}_{text[:50]}.xlsx"
#     file_path = f"/tmp/{file_name}"

#     return wb, file_path

In [ ]:
def create_empty_excel():
    wb = Workbook()
    return wb

In [ ]:
def save_excel_file(wb, chat_id, text, host):
    file_name = f"{chat_id}_{text[:50]}_{host[:(len(host)-3)]}.xlsx"
    file_path = f"/tmp/{file_name}"
    wb.save(file_path)
    return file_path

In [ ]:
def send_excel_file(wb, chat_id, text, host):
    file_path = save_excel_file(wb, chat_id, text, host)
    with open(file_path, "rb") as file:
        bot.send_document(chat_id, file, caption="📗 Вот ваш файл с анализом!")
    os.remove(file_path)

In [ ]:
def add_excel_filter(wb, headers, sz):
  ws = wb.active
  ws.auto_filter.ref = f"A1:{chr(65 + len(headers) - 1)}1"
  # table = Table(displayName="VacanciesTable", ref=f"A1:{chr(65 + len(headers) - 1)}{sz + 1}")
  # ws.add_table(table)
  # style = TableStyleInfo(showFirstColumn=False, showLastColumn=False, showRowStripes=True, showColumnStripes=True)
  # table.tableStyleInfo = style

In [ ]:
def get_list_by_name(values):
  new_list = []
  for v in values:
    new_list.append(v['name'])
  return new_list

In [ ]:
def get_string_list_by_name(values):
  need = get_list_by_name(values)
  if not need:
    return ""
  return "; ".join(need)

In [ ]:
def set_all_borders(ws, data_range):
    thin_border = Border(
        left=Side(border_style="thin"),
        right=Side(border_style="thin"),
        top=Side(border_style="thin"),
        bottom=Side(border_style="thin")
    )

    for row in ws[data_range]:
        for cell in row:
            cell.border = thin_border

In [ ]:
def print_excel_vacancies(wb, vacancies, host = "hh.ru"):
  ws = wb.active
  ws.title = "Вакансии"
  if not vacancies:
    return

  headers = ["ID вакансии", "Вакансия", "Город", "График работы", "Опыт работы",
             "Занятость", "Ночные смены", "Стажировка", "Рабочие часы",
             "Рабочие смены", "Профессиональные роли", "ЗП от", "ЗП до", "Валюта",
             "Тип зарплаты"]
  ws.append(headers)
  for cell in ws[1]:
        cell.font = Font(bold=True)

  for vacancy in vacancies:
        vacancy_url = f"https://{host}/vacancy/{vacancy['id']}"

        id = vacancy['id']
        name = vacancy['name']
        area = vacancy['area']['name']
        graphic_work = vacancy['schedule']['name']
        experience = vacancy['experience']['name']
        employment = vacancy['employment']['name']

        night_shifts = "Есть" if vacancy['night_shifts'] else "Нет"
        internship = "Есть" if vacancy['internship'] else "Нет"

        working_hours = get_string_list_by_name(vacancy['working_hours'])
        work_schedule_by_days = get_string_list_by_name(vacancy['work_schedule_by_days'])
        professional_roles = get_string_list_by_name(vacancy['professional_roles'])

        salary_from = ""
        salary_to = ""
        currency = ""
        gross = ""
        if vacancy['salary'] is not None:
          salary_from = vacancy['salary']['from'] if vacancy['salary']['from'] is not None else ""
          salary_to = vacancy['salary']['to'] if vacancy['salary']['to'] is not None else ""
          currency = vacancy['salary']['currency']
          gross = "До вычета" if vacancy['salary']['gross'] else "На руки"

        ws.append([f'=HYPERLINK("{vacancy_url}", "{id}")',
           name,
           area,
           graphic_work,
           experience,
           employment,
           night_shifts,
           internship,
           working_hours,
           work_schedule_by_days,
           professional_roles,
           salary_from,
           salary_to,
           currency,
           gross
           ])
  rows_count = len(vacancies) + 1
  columns_count = len(headers)
  data_range = f"A1:{chr(65 + columns_count - 1)}{rows_count}"
  set_all_borders(ws, data_range)
  add_excel_filter(wb, headers, len(vacancies))

In [ ]:
def print_excel_dict(wb, sorted_dict, name):
  ws = wb.create_sheet(name)
  wb.active = ws
  headers = [name, "Количество"]
  ws.append(headers)
  for cell in ws[1]:
        cell.font = Font(bold=True)
  for val, c in sorted_dict:
    ws.append([val, c])
  rows_count = len(sorted_dict) + 1
  columns_count = len(headers)
  data_range = f"A1:{chr(65 + columns_count - 1)}{rows_count}"
  set_all_borders(ws, data_range)

In [ ]:
from openpyxl.chart import PieChart, Reference, BarChart, Series
from openpyxl.styles import Font, Border, Side
from openpyxl import Workbook

def add_pie_chart(ws, sorted_dict, chart_title):
    # Создаем pie chart
    pie = PieChart()
    pie.title = chart_title
    labels = Reference(ws, min_row=2, max_row=len(sorted_dict)+1, min_col=1)
    data = Reference(ws, min_row=2, max_row=len(sorted_dict)+1, min_col=2)
    pie.add_data(data, titles_from_data=False)
    pie.set_categories(labels)

    # Размещаем график справа от данных
    ws.add_chart(pie, f"D2")

In [ ]:
def add_bar_chart(ws, sorted_dict, chart_title):
    # Создаем bar chart
    bar = BarChart()
    bar.type = "col"
    bar.title = chart_title
    labels = Reference(ws, min_row=2, max_row=len(sorted_dict)+1, min_col=1)
    data = Reference(ws, min_row=2, max_row=len(sorted_dict)+1, min_col=2)
    bar.add_data(data, titles_from_data=False)
    bar.set_categories(labels)

    # Размещаем график справа от данных
    ws.add_chart(bar, f"D2")

In [ ]:
def print_excel_dict(wb, sorted_dict, name, chart_type='pie'):
    ws = wb.create_sheet(name)
    wb.active = ws
    headers = [name, "Количество"]
    ws.append(headers)
    for cell in ws[1]:
        cell.font = Font(bold=True)

    for val, c in sorted_dict:
        ws.append([val, c])

    rows_count = len(sorted_dict) + 1
    columns_count = len(headers)
    data_range = f"A1:{chr(65 + columns_count - 1)}{rows_count}"
    set_all_borders(ws, data_range)

    # Добавляем график
    if chart_type == 'pie':
        add_pie_chart(ws, sorted_dict, f"Распределение по {name}")
    elif chart_type == 'bar':
        add_bar_chart(ws, sorted_dict, f"Распределение по {name}")

In [ ]:
def print_excel_salary(wb, salary_data):
    ws = wb.create_sheet("Зарплата")
    wb.active = ws

    # Добавляем заголовки
    headers = ["Метрика", "Значение"]
    ws.append(headers)
    for cell in ws[1]:
        cell.font = Font(bold=True)

    # Добавляем данные о зарплате
    salary_metrics = [
        ("Средняя зарплата", salary_data['avg_salary']),
        ("Медианная зарплата", salary_data['median_salary']),
        ("Минимальная зарплата", salary_data['min_salary']),
        ("Максимальная зарплата", salary_data['max_salary'])
    ]

    for metric, value in salary_metrics:
        ws.append([metric, value])

    # Создаем bar chart для распределения зарплат
    if salary_data.get('salary_ranges'):
        ranges_data = sorted(salary_data['salary_ranges'].items())

        # Добавляем данные о диапазонах зарплат
        ws.append([])  # Пустая строка
        ws.append(["Диапазон зарплат", "Количество вакансий"])
        for val, c in ranges_data:
            ws.append([val, c])

        # Добавляем bar chart для диапазонов зарплат
        add_bar_chart(ws, ranges_data, "Распределение вакансий по зарплатным диапазонам")

In [ ]:
def process_salary(vacancies, wb=None):
    if wb is None:
        wb = create_empty_excel()

    print_excel_vacancies(wb, vacancies)
    print_excel_dict(wb, sorted_area_dict, "Город", 'pie')
    print_excel_dict(wb, sorted_employment_dict, "Занятость", 'bar')

## Полезные функции для анализа

In [ ]:
def get_frequency_dict_by_name(vacancies, field):
  print(field)
  frequency_dict = {}
  for v in vacancies:
    frequency_dict[v[field]["name"]] = frequency_dict.get(v[field]["name"], 0) + 1
  return frequency_dict

In [ ]:
def get_frequency_dict_by_field(vacancies, field):
    print(field)
    frequency_dict = {"Есть":0, "Нет":0}
    for v in vacancies:
      if v[field]:
        frequency_dict["Есть"] =frequency_dict.get("Есть", 0) + 1
      else:
        frequency_dict["Нет"] =frequency_dict.get("Нет", 0) + 1
    return frequency_dict

In [ ]:
def get_frequency_dict_by_list_field(vacancies, field):
  frequency_dict = {}
  for v in vacancies:
    if v[field]:
      for value in v[field]:
         frequency_dict[value["name"]] = frequency_dict.get(value["name"], 0) + 1
  return frequency_dict

In [ ]:
def get_sorted_frequenct(vacancies, field, dict_to_sort):
  sorted_frequency = sorted(dict_to_sort.items(), key=lambda item: item[1], reverse=True)
  return sorted_frequency

In [ ]:
def print_top(chat_id, sz, dict_list, msg):
  for i in range(sz):
    name, count = dict_list[i]
    msg += f"{name}: {count}\n"
  bot.send_message(chat_id, msg)

In [ ]:
def process_field_name(wb, chat_id, vacancies, field, symbol = "", top = 10):
  freq = get_frequency_dict_by_name(vacancies, field)
  sorted_frequency = get_sorted_frequenct(vacancies, field, freq)
  print_excel_dict(wb, sorted_frequency, field)

  msg = f"{symbol}\n"
  if (top < len(sorted_frequency)):
    msg += f"🏆 Топ {top} по количеству вакансий:\n"

  top = min(top, len(sorted_frequency))
  print_top(chat_id, top, sorted_frequency, msg)

In [ ]:
def process_binary_field(wb, chat_id, vacancies, field, symbol = ""):
  freq = get_frequency_dict_by_field(vacancies, field)
  sorted_frequency = get_sorted_frequenct(vacancies, field, freq)
  print_excel_dict(wb, sorted_frequency, field)

  msg = f"{symbol}\n"
  print_top(chat_id, len(sorted_frequency), sorted_frequency, msg)

In [ ]:
def process_list_field(wb, chat_id, vacancies, field, symbol = "", top = 5):
  freq = get_frequency_dict_by_list_field(vacancies, field)
  sorted_frequency = get_sorted_frequenct(vacancies, field, freq)
  print_excel_dict(wb, sorted_frequency, field)

  msg = f"{symbol}\n"
  top = min(top, len(sorted_frequency))
  print_top(chat_id, top, sorted_frequency, msg)

## Обработка полных вакансий
1. Description
2. key_skills

In [ ]:
!pip install -U g4f[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 639.7/639.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
import re
from bs4 import BeautifulSoup

def clean_html_description(combined_description):
    # Удаление HTML-тегов
    soup = BeautifulSoup(combined_description, 'html.parser')
    text = soup.get_text(separator=' ', strip=True)

    # Удаление лишних пробелов и переносов строк
    text = re.sub(r'\s+', ' ', text).strip()

    # Удаление специальных символов
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s\.\,\!\?]', '', text)

    return text

In [ ]:
from g4f.client import Client

def process_description(chat_id, full_vacancies, text):

  combined_descriptions = " ".join([vacancy["description"] for vacancy in full_vacancies])
  cleaned_description = clean_html_description(combined_descriptions)

  print(cleaned_description[:100])

  client = Client()

  # Формирование расширенного промпта для создания сводного профиля
  messages = [
      {
          "role": "system",
          "content": """Ты эксперт по подбору персонала и анализу резюме.
          Твоя задача - создать максимально детальный профиль идеального кандидата,
          который объединяет лучшие качества и навыки из предоставленных описаний.

          Алгоритм анализа:
          1. Провести глубокий сравнительный анализ всех представленных профилей
          2. Выделить уникальные и ключевые компетенции
          3. Синтезировать идеальный профиль, который превосходит каждый из представленных

          При составлении профиля учти:
          - Технические навыки и компетенции
          - Профессиональный опыт и достижения
          - Личностные и soft skills
          - Образование и дополнительное обучение
          - Потенциал развития и карьерные амбиции

          Структура профиля:
          🏆 Сводный технический профиль
          🌟 Интегрированные профессиональные компетенции
          👥 Комплексный личностный портрет
          🚀 Стратегия профессионального роста"""
      },
      {
          "role": "user",
          "content": f"""Проанализируй описания кандидатов и создай максимально
          совершенный профиль, который объединяет лучшие характеристики.

          Описания кандидатов:
          {combined_descriptions}

          Задача: Создать эталонный профиль, который превосходит
          каждый из представленных по ключевым параметрам.
          Также не пиши лишние вводные и поясняющие слова, только описание
          Если не получается обобщить, то напиши сама логично подходящие для вакансии {text} пункты профиля идеального кандидата"""
      }
  ]

  try:
      # Создание запроса к модели
      response = client.chat.completions.create(
          model="gpt-4o-mini",  # Можно заменить на другую доступную модель
          messages=messages,
          max_tokens=2000,  # Увеличенное количество токенов для детального анализа
          temperature=0.7  # Баланс креативности и аналитики
      )

      # Извлечение сгенерированного текста
      ideal_candidate_profile = response.choices[0].message.content
      ideal_candidate_profiles[chat_id] = ideal_candidate_profile

      # return f"🏅 Профиль идеального кандидата:\n {ideal_candidate_profile}"
      bot.send_message(chat_id, f"🏅 Профиль идеального кандидата:\n {ideal_candidate_profile}")

  except Exception as e:
      print(f"Ошибка при генерации профиля: {e}")
      return None


In [ ]:
# vacancies = get_vacancies("Программист на Java")
# process_description(1, 1, get_full_vacancies(vacancies) )

In [ ]:
def process_full_vacancies(wb, chat_id, full_vacancies, text = ""):
    print("process_full_vacancies")
    msg = "✨ Ключевые навыки:"
    process_list_field(wb, chat_id, full_vacancies, "key_skills", msg, top=10)
    process_description(chat_id, full_vacancies, text)

## Резюме

если что можно изменить промпт так, чтобы он брал идеального кандидата и на его основе анализ делал)

In [ ]:
# def analyze_resume(chat_id, resume_text):
#     """
#     Анализ резюме пользователя и оценка шансов получения оффера

#     :param chat_id: ID чата пользователя
#     :param resume_text: текст резюме
#     """
#     client = Client()

#     # Промпт для анализа резюме
#     messages = [
#         {
#             "role": "system",
#             "content": """Ты профессиональный HR-эксперт и рекрутер с многолетним опытом.
#             Твоя задача - провести глубокий и объективный анализ резюме кандидата.

#             Критерии оценки:
#             1. Структурированность и качество резюме
#             2. Релевантность опыта работы
#             3. Технические и soft skills
#             4. Потенциал кандидата
#             5. Возможные области улучшения

#             Формат ответа:
#             🎯 Шанс получения оффера: X%

#             📊 Сильные стороны:
#             - Список ключевых преимуществ

#             🔧 Области развития:
#             - Конкретные рекомендации по улучшению резюме и профиля

#             💡 Стратегические советы:
#             - Краткие рекомендации по повышению конкурентоспособности"""
#         },
#         {
#             "role": "user",
#             "content": f"""Проведи детальный анализ следующего резюме:

#             {resume_text}

#             Прошу дать максимально честную и конструктивную оценку,
#             указав реальные шансы на получение оффера и конкретные
#             рекомендации по улучшению профиля."""
#         }
#     ]

#     try:
#         # Создание запроса к модели
#         response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages,
#             max_tokens=1500,
#             temperature=0.7
#         )

#         # Извлечение сгенерированного текста
#         resume_analysis = response.choices[0].message.content

#         # Отправка результатов анализа в чат
#         bot.send_message(chat_id, f"📋 Анализ вашего резюме:\n\n{resume_analysis}")

#     except Exception as e:
#         print(f"Ошибка при анализе резюме: {e}")
#         bot.send_message(
#             chat_id,
#             "Извините, произошла ошибка при анализе резюме. Попробуйте позже."
#         )

In [ ]:
def analyze_resume(chat_id, resume_text, ideal_candidate_profile):
    """
    Анализ резюме пользователя с использованием профиля идеального кандидата

    :param chat_id: ID чата пользователя
    :param resume_text: текст резюме
    :param ideal_candidate_profile: профиль идеального кандидата
    """
    client = Client()

    messages = [
        {
            "role": "system",
            "content": """Ты высококвалифицированный HR-аналитик с глубоким пониманием
            рынка труда и требований работодателей.

            Твоя задача - провести детальный сравнительный анализ
            резюме кандидата с профилем идеального кандидата.

            Критерии оценки:
            1. Соответствие техническим навыкам
            2. Уровень профессионального опыта
            3. Matching soft skills
            4. Потенциал кандидата
            5. Области возможного развития

            Формат ответа:
            🎯 Шанс получения оффера: X%

            ✅ Сильные соответствия профилю:
            - Список ключевых совпадений

            🔍 Расхождения с идеальным профилем:
            - Конкретные недостающие компетенции

            🚀 Рекомендации по развитию:
            - Стратегия закрытия gaps
            - Конкретные шаги для повышения конкурентоспособности"""
        },
        {
            "role": "user",
            "content": f"""Проведи детальный сравнительный анализ:

            Профиль идеального кандидата:
            {ideal_candidate_profile}

            Резюме кандидата:
            {resume_text}

            Задача: Максимально точно оценить соответствие
            кандидата требованиям рынка, указав реальные
            шансы на трудоустройство."""
        }
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=1500,
            temperature=0.7
        )

        resume_analysis = response.choices[0].message.content

        bot.send_message(chat_id, f"📋 Анализ вашего резюме:\n\n{resume_analysis}")

    except Exception as e:
        print(f"Ошибка при анализе резюме: {e}")
        bot.send_message(
            chat_id,
            "Извините, произошла ошибка при анализе резюме. Попробуйте позже."
        )

In [ ]:
# # Пример использования в хендлере бота
# @bot.message_handler(commands=['analyze_resume'])
# def handle_resume_analysis(message):
#     """
#     Хендлер для команды анализа резюме
#     """
#     chat_id = message.chat.id

#     # Проверяем, прикреплено ли резюме
#     if message.document:
#         try:
#             # Скачиваем файл
#             file_info = bot.get_file(message.document.file_id)
#             downloaded_file = bot.download_file(file_info.file_path)

#             # Конвертируем в текст (в зависимости от формата)
#             if message.document.file_name.endswith('.pdf'):
#                 resume_text = extract_text_from_pdf(downloaded_file)
#             elif message.document.file_name.endswith(('.doc', '.docx')):
#                 resume_text = extract_text_from_docx(downloaded_file)
#             else:
#                 resume_text = downloaded_file.decode('utf-8')

#             # Анализируем резюме
#             analyze_resume(chat_id, resume_text)

#         except Exception as e:
#             bot.send_message(
#                 chat_id,
#                 f"Ошибка при обработке файла: {e}"
#             )

#     elif message.text:
#         # Если резюме прислано текстом
#         analyze_resume(chat_id, message.text)

#     else:
#         bot.send_message(
#             chat_id,
#             "Пожалуйста, прикрепите файл резюме или отправьте его текст."
#         )

## Зарплата

In [ ]:
from cachetools import TTLCache
import requests
import xml.etree.ElementTree as ET
from threading import Lock
from datetime import datetime
from statistics import mean, median
from collections import Counter

In [ ]:
class CurrencyConverter:
    def __init__(self, ttl=43.200): #12 часов
        self.cache = TTLCache(maxsize=100, ttl=ttl)
        self.lock = Lock()

    def fetch_exchange_rates(self):
        print("fetching rates")
        today = datetime.now().strftime("%d/%m/%Y")
        url = f"https://cbr.ru/scripts/XML_daily.asp?date_req={today}"
        with self.lock:  # Блокируем доступ для других потоков
            response = requests.get(url)
            if response.status_code != 200:
                raise ValueError("Не удалось получить данные о курсах валют.")

            tree = ET.fromstring(response.content)
            rates = {"RUR": 1.0}  # RUR всегда равен 1.0
            for valute in tree.findall("Valute"):
                char_code = valute.find("CharCode").text
                value = float(valute.find("Value").text.replace(",", "."))
                nominal = int(valute.find("Nominal").text)
                rates[char_code] = value / nominal
            rates["BYR"] = rates["BYN"]
            self.cache.update(rates)

    def get_rate(self, currency):
        if currency not in self.cache:
            print(currency)
            self.fetch_exchange_rates()
        return self.cache.get(currency, None)

In [ ]:
converter = CurrencyConverter()

In [ ]:
def get_converted_salaries(salaries):
  converted_salaries = []
  for salary in salaries:
    currency = salary.get("currency")
    salary_from = salary.get("from")
    salary_to = salary.get("to")

    if currency is None:
      continue

    if currency == "RUR":
      converted_salaries.append(salary)
      continue

    rate = converter.get_rate(currency)
    if rate is None:
        continue

    if salary_from:
        salary_from = salary_from * rate
    if salary_to:
        salary_to = salary_to * rate

    converted_salaries.append({
        "from": salary_from,
        "to": salary_to,
        "currency": "RUR",
        "gross": salary.get("gross")
    })
  return converted_salaries

In [ ]:
def process_specific_salary(chat_id, salaries, text = ""):
  converted_salaries = get_converted_salaries(salaries)
  msg = ""
  if text:
    msg += text
  msg += f"Получено вакансий: {len(converted_salaries)}\n"

  from_values = [s["from"] for s in salaries if s["from"] is not None]
  to_values = [s["to"] for s in salaries if s["to"] is not None]

  min_from = int(round(min(from_values, default=0))) if from_values else None
  max_to = int(round(max(to_values, default=0))) if to_values else None

  mean_from = int(round(mean(from_values))) if from_values else None
  mean_to = int(round(mean(to_values))) if to_values else None

  median_from = int(round(median(from_values))) if from_values else None
  median_to = int(round(median(to_values))) if to_values else None

  avg_min_max = int(round((min_from + max_to) / 2)) if min_from is not None and max_to is not None else None
  avg_mean = int(round((mean_from + mean_to) / 2)) if mean_from is not None and mean_to is not None else None
  avg_meadian = int(round((median_from + median_to) / 2)) if median_from is not None and median_to is not None else None

  if min_from:
      msg += f"Минимальное значение 'от': {min_from:,} ₽\n"
  if max_to:
      msg += f"Максимальное значение 'до': {max_to:,} ₽\n"
  if mean_from:
      msg += f"Среднее значение 'от': {mean_from:,} ₽\n"
  if mean_to:
      msg += f"Среднее значение 'до': {mean_to:,} ₽\n"
  if median_from:
      msg += f"Медиана 'от': {median_from:,} ₽\n"
  if median_to:
      msg += f"Медиана 'до': {median_to:,} ₽\n"

  if avg_mean:
      msg += f"Средняя зарплата: {avg_mean:,} ₽\n"

  bot.send_message(chat_id, msg)

In [ ]:
def process_salary(chat_id, vacancies):
    print("process_salary")
    gross_salary = []
    not_gross_salary = []
    for v in vacancies:
      if v["salary"]:
        if v["salary"]["gross"]:
           gross_salary.append(v["salary"])
        else:
          not_gross_salary.append(v["salary"])

    if gross_salary:
      process_specific_salary(chat_id, gross_salary, "💄👛💍 Анализируем зарплаты до вычета налогов.\n")
    else:
      bot.send_message(chat_id, "Нет вакансий с указаной зарплатов до вычета налогов.")

    if not_gross_salary:
      process_specific_salary(chat_id, not_gross_salary, "💸💵💰 Анализируем зарплаты на руки.\n")
    else:
      bot.send_message(chat_id, "Нет вакансий с указаной зарплатов на руки.")

## Построение частотного словаря для объектов типа "object": {"id", "name"} по name
1. Города.
2. График работы.
3. Опыт работы.
4. Занятость.

In [ ]:
def process_by_name(wb, chat_id, vacancies):
  fields = [
      {
          "field": "area",
          "msg" : "🌃 Города:"
       },
      {
          "field": "schedule",
          "msg" : "📊 График работы:"
       },
      {
          "field": "experience",
          "msg" : "📚 Опыт работы:"
       },
      {
          "field": "employment",
          "msg" : "🔔 Занятость:"
       }
      ]
  for field_data in fields:
    field = field_data["field"]
    msg = field_data["msg"]
    print(field)
    process_field_name(wb, chat_id, vacancies, field, msg)

## Построение частотного словаря для объектов типа "object": true/false
1. Ночные смены
2. Стажировка

In [ ]:
def process_binary_fields(wb, chat_id, vacancies):
  fields = [
      {
          "field": "night_shifts",
          "msg" : "🌙 Ночные смены:"
       },
      {
          "field": "internship",
          "msg" : "👨‍🏫 Стажировка:"
       }
      ]
  for field_data in fields:
    field = field_data["field"]
    msg = field_data["msg"]
    print(field)
    process_binary_field(wb, chat_id, vacancies, field, msg)

## Построение частотного словаря для объектов типа "object": [{id, name}]
1. Рабочие часы.
2. Рабочие смены.
3. Профессиональные роли.

In [ ]:
def process_list_fields(wb, chat_id, vacancies):
  fields = [
      {
          "field": "working_hours",
          "msg" : "⏱ Рабочие часы:"
       },
      {
          "field": "work_schedule_by_days",
          "msg" : "💼 Рабочие смены:"
       },
      {
          "field": "professional_roles",
          "msg" : "🛠 Профессиональные роли:"
       }
      ]
  for field_data in fields:
    field = field_data["field"]
    msg = field_data["msg"]
    print(field)
    process_list_field(wb, chat_id, vacancies, field, msg)

## Запуск анализа

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
user_states = {}
ideal_candidate_profiles = {}

In [ ]:
def perform_analysis(chat_id, vacancies, text, host = "hh.ru"):
  """
    Выполнение анализа вакансий с учетом выбранного хоста

    :param chat_id: ID чата
    :param vacancies: список вакансий
    :param text: текстовое описание
    :param host: хост (по умолчанию hh.ru)
    """
  print(chat_id)
  if not vacancies:
    bot.send_message(
            chat_id,
            "К сожалению, по вашему описанию не найдено ни одной вакансии. 🤔\n"
            "Попробуйте переписать описание вакансии или попробуйте позже — возможно, на сервере временные проблемы."
        )
    return
  print(host)

  wb = openpyxl.Workbook()
  print_excel_vacancies(wb, vacancies, host)

  bot.send_message(chat_id, f"Получено вакансий: {len(vacancies)}")
  bot.send_message(chat_id, "Начинаем анализ найденных вакансий... 🕵️‍♂️")
  with ThreadPoolExecutor() as executor:
      future = executor.submit(get_full_vacancies, vacancies, host)

      process_by_name(wb, chat_id, vacancies)
      process_binary_fields(wb, chat_id, vacancies)
      process_list_fields(wb, chat_id, vacancies)

      process_salary(chat_id, vacancies)

      send_excel_file(wb, chat_id, text, host)
      full_vacancies = future.result()

  process_full_vacancies(wb, chat_id, full_vacancies, text)
  bot.send_message(chat_id, f"Анализ завершен! ✅")

  # Добавляем кнопку для анализа резюме
  markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
  resume_button = types.KeyboardButton("Отправить резюме для анализа")
  markup.add(resume_button)

  bot.send_message(
        chat_id,
        "Хотите проверить шансы получения оффера? 🤔\n"
        "Нажмите кнопку и отправьте свое резюме в текстовом формате.",
        reply_markup=markup
    )

  # Устанавливаем состояние ожидания резюме
  user_states[chat_id] = 'waiting_for_resume'

In [ ]:
# Функция для получения профиля идеального кандидата
# (можно реализовать хранение в словаре или базе данных)
def get_ideal_candidate_profile(chat_id):
    # Здесь должна быть логика получения профиля,
    # сохраненного во время предыдущего анализа
    # Например, можно использовать глобальный словарь или базу данных
    return ideal_candidate_profiles.get(chat_id, "")

In [ ]:
# Обработчик сообщений в боте
@bot.message_handler(func=lambda message: True)
def handle_messages(message):
    chat_id = message.chat.id

    # Проверяем состояние пользователя
    if user_states.get(chat_id) == 'waiting_for_resume':
        # Получаем текст резюме
        resume_text = message.text

        # Получаем профиль идеального кандидата из предыдущего анализа вакансий
        ideal_candidate_profile = get_ideal_candidate_profile(chat_id)

        # Анализируем резюме
        analyze_resume(chat_id, resume_text, ideal_candidate_profile)

        # Сбрасываем состояние
        user_states[chat_id] = None

        # Убираем клавиатуру
        markup = types.ReplyKeyboardRemove()
        bot.send_message(chat_id, "Анализ резюме завершен.", reply_markup=markup)

# TELEGRAM BOT

In [ ]:
!pip install telebot

In [ ]:
import telebot
from telebot import types

In [ ]:
bot = telebot.TeleBot("token")

In [ ]:
# Глобальный словарь для хранения текущего описания вакансии
user_vacancy_context = {}

In [ ]:
@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "Привет! Напиши описание вакансии.")

In [ ]:
@bot.message_handler(func=lambda message: True)
def process_vacancy(message):
    print(message)
    print(20*'-')
    user_input = message.text
    chat_id = message.chat.id

    # Сохраняем описание вакансии для пользователя
    user_vacancy_context[chat_id] = {
        'description': user_input
    }

    # Создаем клавиатуру с хостами
    markup = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    buttons = [types.KeyboardButton(host) for host in hosts]
    markup.add(*buttons)

    # Отправляем сообщение с выбором хоста
    bot.send_message(
        chat_id,
        f"Получено описание вакансии: {user_input}\n\nВыберите сайт для поиска:",
        reply_markup=markup
    )

    # Устанавливаем следующий обработчик
    bot.register_next_step_handler(message, process_host_selection)

def process_host_selection(message):
    chat_id = message.chat.id
    selected_host = message.text

    # Проверяем, что хост валидный
    if selected_host not in hosts:
        bot.send_message(chat_id, "Пожалуйста, выберите хост из предложенных.")
        return

    # Получаем сохраненное описание вакансии
    vacancy_description = user_vacancy_context.get(chat_id, {}).get('description')

    if not vacancy_description:
        bot.send_message(chat_id, "Произошла ошибка. Пожалуйста, начните заново с /start")
        return

    # Убираем клавиатуру
    markup = types.ReplyKeyboardRemove(selective=False)

    # Подтверждаем выбор
    bot.send_message(
        chat_id,
        f"Вы выбрали сайт {selected_host}. Начинаем поиск...",
        reply_markup=markup
    )

    # # Шаг 1: Подтверждение получения описания
    # bot.reply_to(message, f"Получено описание вакансии: {user_input}\nНачинаем обработку...")

    # Шаг 2: Обработки вакансий
    # Получаем вакансии с выбранного хоста
    try:
        vac = get_vacancies(vacancy_description, host=selected_host)

        if not vac:
            bot.send_message(chat_id, "К сожалению, вакансии не найдены.")
            return

        log_message_to_db(message, selected_host, vacancy_description)

        # Анализ вакансий
        perform_analysis(message.chat.id, vac, vacancy_description, host=selected_host)

        url = get_url(vacancy_description, selected_host)
        bot.send_message(chat_id,
                         f"Вы можете посмотреть вакансии по ссылке: [Кликните здесь]({url})",
                         parse_mode='Markdown')
        #здесь будет проверка резюме по кнопке

        # Шаг 3: Сообщение о завершении
        bot.send_message(message.chat.id, "Мы закончили обработку описания вакансии. Можете ввести новое описание.")
    except Exception as e:
        bot.send_message(chat_id, f"Произошла ошибка при поиске: {str(e)}")

    # Очищаем контекст пользователя
    user_vacancy_context.pop(chat_id, None)


In [ ]:
db_connection = mysql.connector.connect(
        host="host",
        user="user",
        password="pass",
        database="db"
    )

if db_connection.is_connected():
        print("Подключение к базе данных успешно установлено.")

In [ ]:
bot.polling()

In [ ]:
if db_connection and db_connection.is_connected():
        db_connection.close()